In [1]:
import torch
import torch.optim as optim
import torch.autograd as autograd
import torch.nn.functional as F
import os
from tqdm import tqdm

In [2]:
# Import our custom modules
from dataset import get_data_loaders
from model import VAE_WGAN

In [3]:
# ================= CONFIGURATION =================
# Hyperparameters
BATCH_SIZE = 64
LR = 5e-5           # Learning Rate
EPOCHS = 100        # Dataset is small (9k), so 100 epochs is reasonable
Z_DIM = 128         # Latent dimension
LAMBDA_GP = 10      # Gradient Penalty weight (Standard WGAN-GP value)
BETA_KL = 1.0       # Weight for KL Divergence
GAMMA_REC = 200.0    # Weight for Reconstruction Loss (High to prioritize accuracy)
ALPHA_ADV = 0.5     # Weight for Adversarial Loss (Texture realism)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
CHECKPOINT_DIR = "checkpoints"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

In [4]:
# ================= HELPER FUNCTIONS =================

def compute_gradient_penalty(D, real_samples, fake_samples):
    """
    Calculates the gradient penalty loss for WGAN-GP.
    Enforces the 1-Lipschitz constraint.
    """
    # Random weight term for interpolation between real and fake samples
    alpha = torch.rand(real_samples.size(0), 1, 1, 1).to(DEVICE)

    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)

    d_interpolates = D(interpolates)

    fake = torch.ones(real_samples.shape[0], 1).to(DEVICE)

    # Get gradient w.r.t. interpolates
    gradients = autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True,
        only_inputs=True,
    )[0]

    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

In [5]:
def loss_function_vae(recon_x, x, mu, logvar):
    """
    Standard VAE Loss = Reconstruction + KL Divergence
    """
    # 1. Reconstruction Loss (L1 or MSE)
    # L1 creates sharper images than MSE
    recon_loss = F.mse_loss(recon_x, x, reduction='mean')

    # 2. KL Divergence
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    kl_div = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())

    return recon_loss, kl_div

### ***================= TRAINING LOOP =================***

In [6]:
# ================= TRAINING LOOP =================
print(f"Training on {DEVICE}...")

Training on cuda...


In [7]:
# 1. Load Data
# Note: We assume 'modis_dataset' folder is in current directory
train_loader, _ = get_data_loaders('modis_dataset_brazil', batch_size=BATCH_SIZE)

Total Normal: 9298
Total Fire: 9299
--- Split Summary ---
Train Set (Normal Only): 7438 images
Test Set (Normal):       1860 images
Test Set (Fire):         1860 images
Total Test Set:          3720 images


In [8]:
# 2. Initialize Model
model = VAE_WGAN().to(DEVICE)

In [9]:
# 3. Optimizers
# We need separate optimizers for the Encoder/Decoder (Generator) and Discriminator
opt_vae = optim.Adam(
    list(model.encoder.parameters()) + list(model.decoder.parameters()),
    lr=LR, betas=(0.5, 0.9)
)
opt_disc = optim.Adam(model.discriminator.parameters(), lr=LR, betas=(0.5, 0.9))

In [10]:
# Training State
global_step = 0

In [11]:
for epoch in range(EPOCHS):
    loop = tqdm(train_loader, leave=True)
    loop.set_description(f"Epoch [{epoch+1}/{EPOCHS}]")

    for batch_idx, (real_imgs, _) in enumerate(loop):
        real_imgs = real_imgs.to(DEVICE)
        curr_batch_size = real_imgs.size(0)

        # ============================================
        #  STEP 1: TRAIN DISCRIMINATOR (The Critic)
        # ============================================
        # In WGAN, we update the critic more often than the generator (e.g., n_critic=5)
        # But for VAE-WGANs, 1:1 is often sufficient. We stick to 1:1 for speed.

        opt_disc.zero_grad()

        # Forward pass VAE to get reconstructed images (Fake)
        recon_imgs, _, _ = model(real_imgs)

        # Detach recon_imgs because we don't want to update VAE weights yet
        fake_imgs = recon_imgs.detach()

        # Critic scores
        real_validity = model.discriminator(real_imgs)
        fake_validity = model.discriminator(fake_imgs)

        # Gradient Penalty
        gp = compute_gradient_penalty(model.discriminator, real_imgs, fake_imgs)

        # WGAN Loss: D(fake) - D(real) + lambda * GP
        # We want to minimize this (which maximizes D(real) - D(fake))
        d_loss = torch.mean(fake_validity) - torch.mean(real_validity) + LAMBDA_GP * gp

        d_loss.backward()
        opt_disc.step()

        # ============================================
        #  STEP 2: TRAIN VAE (Encoder + Decoder)
        # ============================================
        opt_vae.zero_grad()

        # Forward pass
        recon_imgs, mu, logvar = model(real_imgs)

        # VAE Losses
        recon_loss, kl_loss = loss_function_vae(recon_imgs, real_imgs, mu, logvar)

        # Adversarial Loss (Generator tries to fool Critic)
        # We want to minimize -D(fake)
        fake_validity = model.discriminator(recon_imgs)
        adv_loss = -torch.mean(fake_validity)

        # Total VAE Loss
        # We weigh Reconstruction heavily so the Anomaly Detection works well
        vae_loss = (1000.0 * recon_loss) + (BETA_KL * kl_loss) + (ALPHA_ADV * adv_loss)


        vae_loss.backward()
        opt_vae.step()

        # Update Progress Bar
        loop.set_postfix(
            Recon=f"{recon_loss.item():.4f}",
            Critic=f"{d_loss.item():.4f}"
        )

        global_step += 1

    # ================= SAVE CHECKPOINT =================
    # Save every 10 epochs and the final one
    if (epoch + 1) % 10 == 0:
        save_path = os.path.join(CHECKPOINT_DIR, f"vae_wgan_epoch_{epoch+1}.pth")
        torch.save(model.state_dict(), save_path)
        print(f"Saved checkpoint: {save_path}")

Epoch [10/100]: 100%|██████████| 117/117 [00:16<00:00,  7.17it/s, Critic=-4.9042, Recon=0.0141]


Saved checkpoint: checkpoints\vae_wgan_epoch_10.pth


Epoch [20/100]: 100%|██████████| 117/117 [00:26<00:00,  4.46it/s, Critic=-6.1292, Recon=0.0097]


Saved checkpoint: checkpoints\vae_wgan_epoch_20.pth


Epoch [30/100]: 100%|██████████| 117/117 [00:33<00:00,  3.54it/s, Critic=-6.0259, Recon=0.0102]


...

Saved checkpoint: checkpoints\vae_wgan_epoch_80.pth


Epoch [90/100]: 100%|██████████| 117/117 [00:16<00:00,  7.16it/s, Critic=-4.8378, Recon=0.0071]


Saved checkpoint: checkpoints\vae_wgan_epoch_90.pth


Epoch [100/100]: 100%|██████████| 117/117 [00:17<00:00,  6.78it/s, Critic=-3.5842, Recon=0.0082]

Saved checkpoint: checkpoints\vae_wgan_epoch_100.pth


In [12]:
print("Training Complete.")
torch.save(model.state_dict(), "vae_wgan_final.pth")

Training Complete.
